In [1]:
from llama_index import QuestionAnswerPrompt, GPTSimpleVectorIndex, GPTListIndex, LLMPredictor
from langchain import OpenAI
import os
import sys
sys.path.append('../')
import local_secrets as secrets

In [2]:
os.environ['OPENAI_API_KEY'] = secrets.techstyle_openai_key
index_files = [file for file in os.listdir('./stash_index') if file.startswith('DATASCIENCE')]
indexes = [GPTSimpleVectorIndex.load_from_disk(f'./stash_index/{file}') for file in index_files]
[index.set_text(f"stash {file.replace('_', ' ')} repository") for (file,index) in zip(index_files, indexes)]
stash_ds_index = GPTListIndex(indexes)
stash_ds_index.set_text('stash data science repository')
brand_analytics_index = GPTSimpleVectorIndex.load_from_disk('./stash_index/EDW_brand-analytics.json')
ds_techrithm_index =  GPTSimpleVectorIndex.load_from_disk('./stash_index/DATASCIENCE_techrithm.json')

llm_predictor = LLMPredictor(llm=OpenAI(max_tokens=512))
QA_PROMPT_TMPL = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please perform the following: {query_str}\n"
)
QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)

KeyError: 'index_struct'

In [6]:
query_str = "Write code to to get all ShoeDazzle master product id's"
print(ds_techrithm_index.query(query_str, text_qa_template=QA_PROMPT, mode='embedding', llm_predictor=llm_predictor))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 361 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 15 tokens



import local_secrets as secrets
from algorithm_library.product_algorithm import ProductAlgorithm

algo = ProductAlgorithm('ShoeDazzle', 'All', secrets)
df = algo.snowflake_query_to_pandas(algo.queries.q_spid_category())

# Get all ShoeDazzle master product ids
shoedazzle_master_product_ids = df[df['store_group'] == 'ShoeDazzle']['master_product_id'].unique()
print(shoedazzle_master_product_ids)


In [7]:
query_str = "Write a simple query to get Fabletics gross revenue in 2022"
print(brand_analytics_index.query(query_str, text_qa_template=QA_PROMPT, mode='embedding', llm_predictor=llm_predictor))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 681 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 13 tokens



SELECT SUM(fso.cash_gross_revenue) as cash_gross_revenue
FROM edw.analytics_base.finance_sales_ops fso
JOIN edw.data_model.dim_store s on s.store_id = fso.store_id
WHERE store_brand = 'Fabletics'
AND date_object = 'Placed'
AND gender = 'F'
AND store_type IN ('Online','Mobile App')
AND store_region = 'NA'
AND TO_CHAR(fso.date, 'YYYY') = '2022'
GROUP BY month
ORDER BY month;


In [11]:
query_str = "Write a simple query to get Fabletics UK net revenue in 2022"
print(brand_analytics_index.query(query_str, text_qa_template=QA_PROMPT, mode='embedding', llm_predictor=llm_predictor))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 597 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 14 tokens



select sum(product_net_revenue) as product_net_rev 
from edw.analytics_base.finance_Sales_ops fso
join edw.data_model.dim_customer c on c.customer_id = fso.customer_id
join edw.data_model.dim_store st on st.store_id = fso.store_id
join edw.data_model.dim_order_membership_classification omc on omc.order_membership_classification_key = fso.order_membership_classification_key
where st.store_name = 'Fabletics UK'
and st.store_type != 'Retail'
and currency_object = 'usd'
and date_object = 'placed'
and date >= '2022-01-01'
and date <= '2022-12-31'
;
